In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import Planck13 as cosmo
import astropy.units as u
import astropy.constants as const
from scipy.optimize import fsolve
from scipy.interpolate import CubicSpline
from astroquery.sdss import SDSS
from astropy.table import vstack


def j1010_checks(j1010_mul, j1010_ipac):
    """Checks for J1010 OIII bolo, F160 lum, and IR bolo"""
    # OIII luminosity and bolo from OIII
    ja_oiii = 1.2e44 # different from alpaka value??
    i1 = ja_oiii*800/(1+1.3)
    i2 = i1*1.3
    print(f"alpaka OIII_LUM_DERRED: {j1010_mul['OIII_5007_LUM_DERRED'].values[0]:.1e}")
    print(f"paper OIII_LUM_DERRED {ja_oiii:.1e}")
    print(f"by paper OIII, my core 1 bolometric: {i1:.1e}, my core 2 bolometric: {i2:.1e}")

    # F160 luminosity 
    f160 = 3.23e-12*u.erg/u.s/(u.cm)**2 # from paper
    ja_ir = 1.1e11
    da = cosmo.luminosity_distance(j1010_mul['Z'].values[0])
    print(f"paper F160 IR lum: {ja_ir:.1e}")
    print(f"my F160 IR Lum: {(f160*4*np.pi*da**2).to(u.L_sun):.1e}")

    # IR bolo from WISE
    paper_ir_bol = 6e46
    print(f"paper bolometric luminosity from WISE IR: {paper_ir_bol:.2e}")
    
    wmag10, wmagerr10  = get_wise_mags(j1010_ipac)
    j10_w_lum = wise_lum_from_mag(wmag10[:,0], wmagerr10[:,0], 22, 0.198).value
    j10_ir_bol = j10_w_lum*10**1.12#10**(spl(np.log10(j10_w_lum)))
    print(f"J1010 AGN lum from WISE IR at 22 microns: {j10_ir_bol:.2e}")


def get_wise_mags(wise_):
    """get wise mags and mag errors from data frame of ipac search results"""
    ## get wise mags and errors
    w1mag = wise_['w1mpro']
    w2mag = wise_['w2mpro']
    w3mag = wise_['w3mpro']
    w4mag = wise_['w4mpro']
    wmags_ = np.array([w1mag, w2mag, w3mag, w4mag])
    wmags_err_ = np.array([wise_['w1sigmpro'], wise_['w2sigmpro'], wise_['w3sigmpro'], wise_['w4sigmpro']])
    return wmags_, wmags_err_


def wise_lum_from_mag(wmags_, wmags_err_, obs_wavelength_, redshift_):
    """calculate wise luminosity from magnitude at some observed wavelength"""
    ## change mags to fluxes -- http://wise2.ipac.caltech.edu/docs/release/allsky/expsup/sec4_4h.html#example
    zeromagflux = np.array([309.540, 171.787, 31.674, 8.363])*u.Jy
    fluxdens = zeromagflux*10**(-wmags_/2.5) # in Jy
    # now either interpolate flux dens to some wavelength or use a band from wise
    wise_wavelengths = np.array([3.4, 4.6, 12., 22.]) # 1e-6 m
    if np.isin(obs_wavelength_, wise_wavelengths): # check if need to interpolate to non-wise wl
        obs_flux = fluxdens[wise_wavelengths==obs_wavelength_][0]
    else: # interpolate
        fluxdens_err = zeromagflux*10**(-wmags_err_/2.5)
        ## interpolate - use straight line
        wiseflux = np.polyfit(wise_wavelengths, fluxdens.value,1, w=1./fluxdens_err)
        ## get flux at obs wavelength, i.e. just a straight line here
        obs_flux = (wiseflux[0]*obs_wavelength_+wiseflux[1])*u.Jy      
    ## change to luminosity
    obs_hz = (const.c/(obs_wavelength_*u.micron)).to(u.Hz)
    lum = (obs_flux*obs_hz*4*np.pi*
           cosmo.luminosity_distance(redshift_)**2).to(u.erg/u.s)
    return lum


def correct_ir():
    """correct IR luminosity at 15 microns rest frame based on Hopkins+20"""
    # load hopkins bolometric correction
    with open("/home/insepien/research-data/pop-result/bc.txt","r") as f:
        d = f.read().splitlines()
    hopkins = pd.DataFrame([d[1:][i].split(' ') for i in range(len(d[1:]))],columns=d[0].split(' '))
    Lbol = np.array(list(hopkins['Lbols'].values), dtype=float)
    LIR = np.array(list(hopkins['LIRs'].values), dtype=float)
    spl = CubicSpline(LIR, Lbol)
    return spl

def get_wise_ir_lums(wise_, alpaka_,wise_key = 'designation', mul_key='Desig',wl_=22):
    """calculate wise IR luminosity and bolometric lum, 
        default keys are for magellan sample"""
    ## get wise mags and errors
    wmags, wmags_err_nan = get_wise_mags(wise_)
    # replace nan values in mag error with median
    wmags_err = np.nan_to_num(wmags_err_nan,np.median(wmags_err_nan))
    # calculate luminosity
    wise_lums = np.zeros((len(wise_)))
    for i in range(0, len(wise_)):
        on = wise_.loc[i,wise_key]
        z = np.mean(alpaka_[alpaka_[mul_key]==on]['Z'])
        wise_lums[i] = wise_lum_from_mag(wmags[:,i], wmags_err[:,i], wl_, z).value
    # check wavelength to see how to do bolo correction
    if wl_==15: # use Hopkins+2020 if at 15 microns
        spl = correct_ir()
        irbol = 10**(spl(np.log10(wise_lums)))
    else: # else correct by 12%
        irbol = wise_lums*10**1.12
    return wmags, wise_lums,irbol

def make_desig(data, ra_key='ra', dec_key='dec'):
    """make designation if df has 'ra' and 'dec' columns"""
    desig=[]
    for posstring in SkyCoord(data[ra_key].values*u.deg, data[dec_key].values*u.deg).to_string("hmsdms"):
        posstring = posstring.split(' ')
        des_ra = posstring[0][0:2]+posstring[0][3:5]
        des_dec = posstring[1][0:3]+posstring[1][4:6]
        desig.append('J'+des_ra+des_dec)
    return desig


def search_sdss(cat,query_field=['ra','dec','z','spectroFlux_r']):
    """query sdss specobj with ra and dec given some catalog"""
    search_results = []
    # search some radius in SDSS
    for i in cat['RA'].index.values:
        pos = SkyCoord(cat['RA'][i]*u.deg, cat['DEC'][i]*u.deg)
        xid = SDSS.query_region(pos, radius='2 arcsec',specobj_fields=query_field)
        search_results.append(xid)
    # put rows into table
    sdss_matches = vstack(search_results).to_pandas()
    # make designations
    sdss_matches['DESIG'] = make_desig(sdss_matches)
    # examined dups and can take first values, so drop other dup rows
    nodup_mask = sdss_matches['DESIG'].drop_duplicates(keep='first').index.values
    sdss_nodups = sdss_matches.loc[nodup_mask]
    return sdss_nodups



In [ ]:
# load alpaka
alpaka = Table(fits.getdata('/home/insepien/research-data/alpaka/ALPAKA_v1_withDes.fits')).to_pandas()
# read 171 sample
mul171 = pd.read_pickle("/home/insepien/research-data/alpaka/mull171.pkl")
mul171['DESIG'] = make_desig(mul171, ra_key='RA', dec_key='DEC')
# get j1010 row
j10_mul = alpaka[alpaka['Desig']=="J1010+1413"]
# j1010 WISE search
j10_ipac = pd.read_csv("/home/insepien/research-data/alpaka/j1010_ipac.csv")

### get magellan & HST wise lum and r-band mag

In [ ]:
################# magellan
# this file is from query of sources on WISE, instructions for query in kris candidate selection notebook
# wise search has 39 objects fitted in magellan sample
wise39 = pd.read_pickle("/home/insepien/research-data/alpaka/wise_39fits.pkl")
# read alpaka for optical comparison and redshift
# alpaka has 41 rows, since J0926+0724 (1 is non-agn) and J1222-0007 (dual) are duplicated
alpaka39 = pd.read_pickle("/home/insepien/research-data/alpaka/alpaka_39fits.pkl")
alpaka39.reset_index(inplace=True)
# get magellan mag and lum
wise_magel_mag, wise_magel_lum, magel_irbol = get_wise_ir_lums(wise39,alpaka39)
#################### OIII
oiii = alpaka39['OIII_5007_LUM_DERRED']

################# HST
# read wise search
wise171 = pd.read_csv("/home/insepien/research-data/alpaka/wise_mul.csv")
desig = [wise171['designation'][i][:5]+wise171['designation'][i][10:15] for i in range(len(wise171))]
wise171['DESIG'] = desig
# find duplicated rows
dupmask = wise171['DESIG'].duplicated(keep=False)
# make separate df
cols = ['DESIG',"ra","dec","ra_01","dec_01"]
dups = wise171[dupmask][cols].copy()
# add cols of ra and dec differences 
dups['del_ra'] = (dups['ra']-dups['ra_01']).abs()
dups['del_dec'] = (dups['dec']-dups['dec_01']).abs()
# group df by name and find row index with lowest del_ra and del_dec and check
min_dels = dups.groupby(by='DESIG').idxmin()
if np.sum(min_dels['del_ra']==min_dels['del_dec'])==0:
    keep_ind = min_dels['del_ra'].values
else:
    print("min del_ra does not match min del_dec")
# turns out it is ok to do this
keep_ind = min_dels['del_ra'].values
# drop duplicated rows except for row with lowest ra and dec difference
drop_ind = dups.index[~dups.index.isin(keep_ind)]
wise171.drop(drop_ind,inplace=True)
wise171.reset_index(inplace=True)
#get luminosities
hst_wise_mags, hst_wise_lums, hst_irbol = get_wise_ir_lums(wise171,mul171,wise_key='DESIG',mul_key='DESIG',wl_=22)
# get OIII lum
inf_mask = (mul171['OIII_5007_LUM_DERRED']>-np.inf) & (mul171['OIII_5007_LUM_DERRED']<np.inf)
oiii171 = mul171['OIII_5007_LUM_DERRED'][inf_mask]
# j1010 numbers
j10_ipac['DESIG'] = make_desig(j10_ipac)
j10_w_mag,j10_w_lum,j10_ir_bol = get_wise_ir_lums(j10_ipac,j10_mul,wise_key="DESIG",wl_=22)

################# r-band
j10_r_magab = 16.83
# read query results and calculate ab mag
sdss = pd.read_pickle("~/research-data/alpaka/sdss_rband_171.pkl")
hst_magab = 22.5 - 2.5 * np.log10(sdss['spectroFlux_r'])


In [ ]:
fig,ax = plt.subplots(2,2,figsize=(13,10))
ax[0,0].hist(wise171['w4mpro']+6.620,label='hst IR',color='green')
ax[0,0].hist(wise_magel_mag[3,:]+6.620, label='magellan IR', color='orange') 
ax[0,0].axvline(j10_ipac['w4mpro'].values[0]+6.620, label='J1010 IR',c='blue')
ax[0,0].hist(hst_magab,label='hst SDSS-R',color='red')
ax[0,0].axvline(j10_r_magab,c='red',label='j1010 SDSS-R')
ax[0,0].set_title("Magnitude")
ax[0,0].legend()
ax[0,0].set_xlabel("AB Mag at 22 microns")

ax[0,1].hist(np.log10(hst_wise_lums),label='hst IR',color='green')
ax[0,1].hist(np.log10(wise_magel_lum),label='magellan IR',color='orange')
ax[0,1].axvline(np.log10(j10_w_lum),label='j1010 IR',color='blue')
ax[0,1].hist(np.log10(oiii171),label='hst OIII_DERRED',color='red',histtype='step')
ax[0,1].axvline(np.log10(j10_mul['OIII_5007_LUM_DERRED'].values[0]),color='r',label='j1010 OIII_DERRED')
ax[0,1].set_xlabel("Log(Luminosity) [ergs/s]")
ax[0,1].set_title("Luminosity")
ax[0,1].legend()

ax[1,0].hist(np.log10(hst_irbol),label='HST IR', color='green')
ax[1,0].hist(np.log10(magel_irbol),label='Magellan IR', color='orange')
ax[1,0].axvline(np.log10(j10_ir_bol),label='j1010 IR',c="orange")

ax[1,0].hist(np.log10(oiii*800),label="OIII*800",color='steelblue',histtype='step')
ax[1,0].axvline(np.log10(1.2e43*800),label='j1010 OIII*800',c="steelblue")

ax[1,0].set_xlabel("Log(L_bol)")
ax[1,0].set_ylabel("# quasars")
ax[1,0].set_title("bolometric luminosity with OIII correction of 800")
ax[1,0].legend()

ax[1,1].hist(np.log10(hst_irbol),label='HST IR', color='green')
ax[1,1].hist(np.log10(magel_irbol),label='Magellan IR', color='orange')
ax[1,1].axvline(np.log10(j10_ir_bol),label='j1010 IR',c="orange")

ax[1,1].hist(np.log10(oiii*100),label="OIII*100",color='steelblue',histtype='step')
ax[1,1].axvline(np.log10(1.2e43*100),label='j1010 OIII*100',c="steelblue")

ax[1,1].set_xlabel("Log(L_bol)")
ax[1,1].set_ylabel("# quasars")
ax[1,1].set_title("bolometric luminosity with OIII correction of 100")
ax[1,1].legend()
fig.tight_layout()
;


In [ ]:
fig,ax = plt.subplots(2,2,figsize=(13,10))
ax[0,0].hist(wise171['w4mpro']+6.620,label='hst sample IR',color='green')
ax[0,0].hist(pd.DataFrame(wise_magel_mag).loc[3]+6.620, label='magellan sample IR', color='orange') 
ax[0,0].axvline(j10_ipac['w4mpro'].values[0]+6.620, label='J1010 IR',c='blue')
ax[0,0].hist(hst_magab,label='hst sample SDSS-R',color='red')
ax[0,0].axvline(j10_r_magab,c='red',label='j1010 SDSS-R')
ax[0,0].set_title("Magnitude in WISE IR and SDSS R-band")
ax[0,0].legend()
ax[0,0].set_xlabel("AB Mag")

# ax[0,1].hist(np.log10(hst_wise_lums),label='hst IR',color='green')
# ax[0,1].hist(np.log10(wise_magel_lum),label='magellan IR',color='orange')
ax[0,1].hist(np.log10(mul171['OIII_5007_LUM']+mul171['OIII_5007B_LUM']),label='narrow+broad [OIII] luminosity')
ax[0,1].axvline(np.log10(j10_mul['OIII_5007_LUM']+j10_mul['OIII_5007B_LUM']).values[0],label='J1010 n+b [OIII] luminosity')
ax[0,1].hist(np.log10(oiii171),label='OIII_DERRED',color='red')
ax[0,1].axvline(np.log10(j10_mul['OIII_5007_LUM_DERRED'].values[0]),color='r',linestyle='--',label='j1010 OIII_DERRED')
ax[0,1].set_xlabel("Log(Luminosity) [ergs/s]")
ax[0,1].set_title("Compare [OIII] and [OIII] de-reddened luminosities")
ax[0,1].legend()
[ax[0,i].set_xlabel("Number of AGNs") for i in range(2)]

ax[1,0].hist(np.log10(hst_wise_lums),label='hst sample',color='green')
ax[1,0].hist(np.log10(wise_magel_lum),label='magellan sample',color='orange')
ax[1,0].axvline(np.log10(j10_w_lum),label='j1010 IR',color='blue')
ax[1,0].set_xlabel("Log(IR Luminosity) [ergs/s]")
ax[1,0].set_title("WISE 22-micron Luminosity")
ax[1,0].legend(loc='upper left')

ax[1,1].hist(np.log10(hst_irbol),label='HST IR', color='green')
ax[1,1].hist(np.log10(magel_irbol),label='Magellan IR', color='orange')
ax[1,1].axvline(np.log10(j10_ir_bol),label='j1010 IR',c="blue")

ax[1,1].set_xlabel("Log(L_bol)")
ax[1,1].set_title("Bolometric luminosity (12% correction on IR lum)")
ax[1,1].legend(loc='upper left')
framenum=np.arange(1,5)
axx = ax.ravel()
[axx[i].text(0.9, 0.9, f"{framenum[i]}", transform=axx[i].transAxes, fontsize=30, 
            fontweight="bold", va="top", ha="left") for i in range(4)]
fig.tight_layout();

## new sample

In [ ]:
# cut only in z and type from alpaka and do wise search
mask = (alpaka['Z']>0.1) & (alpaka['Z']<0.5) & (alpaka['AGN_TYPE']==2)
alpaka_z05 = alpaka[mask]
# to make ipac .dat query file
# ipac_cat = Table([alpaka[mask]['RA'], alpaka[mask]['DEC']], names=['ra','dec'])
# for_wise_search = ascii.write(ipac_cat, 'wise_search_z05.dat', format='ipac', overwrite=True);
# read wise search results for all type 2 under z=0.5
wsearch = pd.read_csv('wise_z05_result.csv')
wsearch['DESIG'] = make_desig(wsearch,ra_key='ra_01',dec_key='dec_01')
# calculate mag and lum
wmag,wlum,wbol = get_wise_ir_lums(wsearch, alpaka_z05,wise_key='DESIG',mul_key='Desig',wl_=22)

In [ ]:
# cut only in z and type from alpaka and do wise search
mask = (alpaka['Z']>0.1) & (alpaka['Z']<0.5) & (alpaka['AGN_TYPE']==2)
alpaka_z05 = alpaka[mask]
# to make ipac .dat query file
# ipac_cat = Table([alpaka[mask]['RA'], alpaka[mask]['DEC']], names=['ra','dec'])
# for_wise_search = ascii.write(ipac_cat, 'wise_search_z05.dat', format='ipac', overwrite=True);
# read wise search results for all type 2 under z=0.5
wsearch = pd.read_csv('wise_z05_result.csv')
wsearch['DESIG'] = make_desig(wsearch,ra_key='ra_01',dec_key='dec_01')
# calculate mag and lum
wmag,wlum,wbol = get_wise_ir_lums(wsearch, alpaka_z05,wise_key='DESIG',mul_key='Desig',wl_=22)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,3))

ax[0].hist(wmag[3,:]+6.620,histtype='step')
ax[0].axvline(j10_ipac['w4mpro'].values[0]+6.620,label='j1010')

ax[1].hist(np.log10(wbol[(wbol<np.inf)&(wbol>-np.inf)]),histtype='step')
ax[1].axvline(np.log10(j10_ir_bol),label='j1010')

ax[0].set_xlabel("wise AB mag at $22\mu m$")
ax[1].set_xlabel("Log(Lbol) [erg/s]")
ax[0].set_title("WISE magnitude of type-2 AGN in z=0.1-0.5")
ax[1].set_title("Bolometric luminosity\n(12% correction to $Log(L_{22 \mu m})$) ")

[ax[i].legend(loc='upper right') for i in range(2)];

In [ ]:
key = 'HA_FLUX'
# check that the log(lbol)>46 subset makes sense in OIII
mask_L46 = alpaka_z05['Desig'].isin(wsearch[np.log10(wbol)>46]['DESIG'])
mul89 = alpaka_z05[mask_L46]
L46 = mul89[key][(mul89[key]>0) & (np.isfinite(mul89[key]))]
plt.hist(L46)
plt.axvline(np.log10(j10_mul[key].values[0]))

In [ ]:
plt.hist(alpaka_z05[mask_L46]['Z'])
plt.xlabel("Z")
plt.ylabel("# AGN")